In [1]:
import struct
import os
import scipy.io
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout

In [3]:
ECG_data = []

for idx in range(1, 81) : #0~89까지 bin file 순회
    data_path = os.path.join('ECG_' + str(idx) + '.bin') #file 경로 지정
    with open(data_path, "rb") as f : # .bin file을 os 함수를 이용하여 rb로 open
        data = np.frombuffer(f.read(), dtype=np.float16) # bin file 내용 저장
    data = np.append(data, np.array([0]*(5051128 - len(data)))) # 길이가 짧은 것들은 0으로 채우기
    ECG_data.append(data) # dataframe을 만들기 위해 data를 ECG_data에 추가

    f.close() #읽은 .bin file close
    
df_col = [i for i in range(0, 5051128)] #dataframe의 column

ECG = pd.DataFrame(columns = df_col)

for i in range(0, 80) :
    ECG.loc[i] = ECG_data[i]
ECG

,0,1,2,3,4,5,6,7,8,9,...,5051118,5051119,5051120,5051121,5051122,5051123,5051124,5051125,5051126,5051127
0,0.000012,0.002014,0.000856,0.031860,0.000187,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000012,0.002014,0.001469,0.635254,0.000187,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000012,0.002014,0.001469,0.382812,0.000191,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000012,0.002014,0.000061,0.000063,0.000185,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000012,0.002014,0.001713,0.009918,0.000187,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.000012,0.002014,0.019623,0.000155,0.000189,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76,0.000012,0.002014,0.019623,0.000032,0.000191,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,0.000012,0.002014,0.023529,0.504395,0.000184,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0.000012,0.002014,0.023529,0.000500,0.000187,0.000015,0.00174,0.009781,0.131958,0.324463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
ECG_data = []
sub_num = [1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 38, 39, 41, 42, 43, 44, 45, 47, 48, 52, 53, 54, 55, 59, 59, 60, 62, 63, 64, 65, 66, 68, 70, 72, 73, 74, 75, 76, 77, 78]
for idx in range(1, 81) : #0~89까지 bin file 순회
    if idx in sub_num :
        data_path = os.path.join('ECG_' + str(idx) + '.bin') #file 경로 지정
        with open(data_path, "rb") as f : # .bin file을 os 함수를 이용하여 rb로 open
            data = np.frombuffer(f.read(), dtype=np.float16) # bin file 내용 저장
    
        ECG_data.append(np.array(data))

In [3]:
len(ECG_data)

59

In [4]:
def replace_nan_with_mean(arr):
    nan_indices = np.isnan(arr)
    
    if nan_indices[0]:
        arr[0] = arr[1]
    
    if nan_indices[-1]:
        arr[-1] = (arr[-2] + arr[-3]) / 2
    
    for i in range(1, len(arr) - 1):
        if nan_indices[i]:
            arr[i] = (arr[i-1] + arr[i-2]) / 2
    
    return arr

In [6]:
for i in range(0, len(ECG_data)) :
    ECG_data[i] = replace_nan_with_mean(ECG_data[i])

In [8]:
for i in range(0, len(ECG_data)) :
    print(np.isnan(ECG_data[i]))

[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False

In [9]:
movie_feel = pd.read_csv('./movies.csv')

In [10]:
scores = pd.read_csv('./score.csv')

In [11]:
movie_feel

,First,Second,Third,Fourth,Fifth,Sixth
0,neutral,disgust,fear,sad,happy,anger
1,anger,disgust,neutral,happy,sad,fear
2,happy,neutral,anger,sad,disgust,fear
3,disgust,happy,sad,anger,neutral,fear
4,neutral,anger,disgust,happy,sad,fear
...,...,...,...,...,...,...
84,neutral,sad,happy,fear,disgust,anger
85,anger,sad,neutral,fear,happy,disgust
86,sad,anger,disgust,fear,neutral,happy
87,disgust,sad,anger,happy,fear,neutral


In [12]:
scores.drop([3, 7, 13, 15, 33, 34, 36, 39, 45, 48, 49, 50, 55, 56, 57, 60, 66, 68, 70, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88], axis = 0, inplace = True)

In [13]:
vid1 = scores[['Video1_neutral', 'Video1_fear', 'Video1_sad', 'Video1_happy', 'Video1_anger', 'Video1_disgust']]
vid2 = scores[['Video2_neutral', 'Video2_fear', 'Video2_sad', 'Video2_happy', 'Video2_anger', 'Video2_disgust']]
vid3 = scores[['Video3_neutral', 'Video3_fear', 'Video3_sad', 'Video3_happy', 'Video3_anger', 'Video3_disgust']]
vid4 = scores[['Video4_neutral', 'Video4_fear', 'Video4_sad', 'Video4_happy', 'Video4_anger', 'Video4_disgust']]
vid5 = scores[['Video5_neutral', 'Video5_fear', 'Video5_sad', 'Video5_happy', 'Video5_anger', 'Video5_disgust']]
vid6 = scores[['Video6_neutral', 'Video6_fear', 'Video6_sad', 'Video6_happy', 'Video6_anger', 'Video6_disgust']]
vid1.loc[1].values

array([1, 3, 3, 1, 6, 6], dtype=int64)

In [14]:
matsub_num = ['001', '002', '003', '005', '006', '007', '009', '010', '011', '012', '013', '015', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '036', '038', '039', '041', '042', '043', '044', '045', '047', '048', '052', '053', '054', '055', '059', '060', '062', '063', '064', '065', '066', '068', '070', '072', '073', '074', '075', '076', '077', '078']
len(matsub_num)

59

In [15]:
seg_label = []
for num in matsub_num :
    mat_path = os.path.join('./ECG_experiment_matfile', num, 'selected_data')
    #print(mat_path)
    mat_file = scipy.io.loadmat(mat_path)
    seg_label.append([mat_file['Segment1'][0], vid1.loc[0].values])
    seg_label.append([mat_file['Segment2'][0], vid2.loc[0].values])
    seg_label.append([mat_file['Segment3'][0], vid3.loc[0].values])
    seg_label.append([mat_file['Segment4'][0], vid4.loc[0].values])
    seg_label.append([mat_file['Segment5'][0], vid5.loc[0].values])
    seg_label.append([mat_file['Segment6'][0], vid6.loc[0].values])

#x = mat_file['Segment1'].reshape(1, -1)
print(seg_label[0])
#len(mat_file['Segment1'][0])

[array([507, 503, 503, ..., 492, 493, 494], dtype=uint16), array([7, 1, 1, 5, 1, 1], dtype=int64)]


In [100]:
X_train = []
y_true = []
for xt in seg_label :
    print(xt[0].shape)#.reshape(-1).shape)
    X_train.append(tf.convert_to_tensor(xt[0]))
    y_true.append(tf.convert_to_tensor(xt[1]))

(178490,)
(193736,)
(191559,)
(159628,)
(226382,)
(214049,)
(285106,)
(258045,)
(237750,)
(301537,)
(280274,)
(249346,)
(172727,)
(136188,)
(163316,)
(160548,)
(147815,)
(142832,)
(248543,)
(298048,)
(269761,)
(315226,)
(292998,)
(260666,)
(140709,)
(170159,)
(158161,)
(134164,)
(145618,)
(160889,)
(181343,)
(196965,)
(211908,)
(175231,)
(167081,)
(200361,)
(134351,)
(161111,)
(158381,)
(144727,)
(141451,)
(170396,)
(172779,)
(208135,)
(193459,)
(164107,)
(196794,)
(176780,)
(133879,)
(149905,)
(152488,)
(127160,)
(161277,)
(136982,)
(188084,)
(159751,)
(156134,)
(177836,)
(174822,)
(148297,)
(206909,)
(172541,)
(181659,)
(203402,)
(185867,)
(218833,)
(193425,)
(152510,)
(182887,)
(160569,)
(179787,)
(164289,)
(253954,)
(233979,)
(245393,)
(280585,)
(275829,)
(296755,)
(159708,)
(139677,)
(157000,)
(144549,)
(133180,)
(168911,)
(159952,)
(143686,)
(133383,)
(169169,)
(157241,)
(140432,)
(324760,)
(268552,)
(307066,)
(256062,)
(277920,)
(301863,)
(194011,)
(169678,)
(161786,)
(175598,)


In [101]:
max_length = 338915
padded_data = np.zeros((len(X_train), max_length))

for i, d in enumerate(X_train):
    padded_data[i, :len(d)] = d

print(padded_data.shape)
X_train = tf.convert_to_tensor(padded_data)
X_train

(354, 338915)


<tf.Tensor: shape=(354, 338915), dtype=float64, numpy=
array([[507., 503., 503., ...,   0.,   0.,   0.],
       [535., 551., 571., ...,   0.,   0.,   0.],
       [500., 500., 498., ...,   0.,   0.,   0.],
       ...,
       [593., 606., 612., ...,   0.,   0.,   0.],
       [469., 469., 471., ...,   0.,   0.,   0.],
       [487., 483., 482., ...,   0.,   0.,   0.]])>

In [102]:
y_true
y_true = tf.convert_to_tensor(y_true)
y_true

<tf.Tensor: shape=(354, 6), dtype=int64, numpy=
array([[7, 1, 1, 5, 1, 1],
       [1, 5, 1, 1, 1, 7],
       [1, 6, 1, 1, 1, 1],
       ...,
       [2, 1, 3, 1, 1, 1],
       [2, 1, 1, 6, 1, 1],
       [1, 1, 1, 1, 6, 3]], dtype=int64)>

In [112]:
X_train.shape[1]

338915

In [103]:
# 데이터의 형태 확인
input_shape = X_train.shape[1:]  # (338915,)
num_classes = y_true.shape[1]  # 6

X_train = np.expand_dims(X_train, axis=-1)

# 1D-CNN 모델 생성
model = tf.keras.models.Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPool1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_true, batch_size=32, epochs=10, validation_split=0.2)

ValueError: Input 0 of layer "conv1d_10" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 338915)

In [113]:
cnn_model = tf.keras.models.Sequential()
#First CNN layer  with 32 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (X_train.shape[1],1)))
#Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Third CNN layer with 128 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=128, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Fourth CNN layer with Max pooling
cnn_model.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
cnn_model.add(Dropout(0.5))
#Flatten the output
cnn_model.add(Flatten())
#Add a dense layer with 256 neurons
cnn_model.add(Dense(units = 256, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 512, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Softmax as last layer with five outputs
cnn_model.add(Dense(units = 5, activation='softmax'))

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[21690624,256] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:Mul]

In [109]:
cnn_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [106]:
cnn_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_11 (Conv1D)          (None, 354, 32)           128       
                                                                 
 conv1d_12 (Conv1D)          (None, 354, 64)           6208      
                                                                 
 conv1d_13 (Conv1D)          (None, 354, 128)          24704     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 177, 128)         0         
 1D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 177, 128)          0         
                                                                 
 flatten_2 (Flatten)         (None, 22656)             0         
                                                     

In [107]:
cnn_model_history = cnn_model.fit(X_train, y_true, epochs=10, batch_size = 10)#, validation_data = (X_test, y_test))

Epoch 1/10


ValueError: in user code:

    File "C:\Users\82108\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\82108\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\82108\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\82108\anaconda3\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\82108\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\82108\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 o

In [ ]:
plt.plot(cnn_model_history.history['accuracy'])
plt.plot(cnn_model_history.history['val_accuracy'])
plt.legend(["accuracy","val_accuracy"])
plt.title('Accuracy Vs Val_Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [ ]:
plt.plot(cnn_model_history.history['loss'])
plt.plot(cnn_model_history.history['val_loss'])
plt.legend(["loss","val_loss"])
plt.title('Loss Vs Val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')